In [2]:
import os
import numpy as np

import jdaviz
from jwst import datamodels

import specutils
from astropy.io import fits
from astropy import units as u
from astropy import table
from astropy import nddata

from astroquery.mast import Observations

In [3]:
Observations.login(open(os.path.expanduser('~/.mast_token')).read().strip())

INFO: MAST API token accepted, welcome Erik Tollerud [astroquery.mast.auth]


In [5]:
obses = Observations.query_criteria(proposal_id=2609)
nsobses = obses[obses['instrument_name']=='NIRSPEC']
data_products = Observations.get_product_list(nsobses)
data_products[data_products['calib_level']==2]

KeyboardInterrupt: 

In [ ]:
results = Observations.download_products(data_products[np.in1d(data_products['productSubGroupDescription'], ['X1D', 'S2D'])])
results

In [8]:
ls mastDownload/JWST/jw02609007001_03101_00003_nrs2/

jw02609007001_03101_00003_nrs2_s2d.fits
jw02609007001_03101_00003_nrs2_x1d.fits


In [10]:
#x1df = fits.open(results['Local Path'][-1])
x1df = fits.open('mastDownload/JWST/jw02609007001_03101_00003_nrs2/jw02609007001_03101_00003_nrs2_x1d.fits')

In [13]:
metas1d = []
specs1d = []
for hdu in x1df[1:-1]:
    qt = table.QTable.read(hdu)
    metas1d.append(qt.meta)
    spec = specutils.Spectrum1D(spectral_axis=qt['WAVELENGTH'], 
                         flux=qt['FLUX'], 
                         uncertainty = nddata.StdDevUncertainty(qt['FLUX_ERROR']),
                         meta=qt.meta,
                               )
    specs1d.append(spec)

In [14]:
mv = jdaviz.Mosviz()

mv.load_1d_spectra(specs1d, None)
mv.add_column('source name', [meta['SRCNAME'] for meta in metas1d])

mv.add_column('RA', [meta['SRCRA'] for meta in metas1d])
mv.add_column('Dec', [meta['SRCDEC'] for meta in metas1d])

mv.show('sidecar:tab-after')

In [18]:
s2ddm = datamodels.open(x1df.filename().replace('x1d', 's2d'))

specs2d = []
for s in s2ddm.slits:
    sc = specutils.Spectrum1D(flux=s.data*u.Unit(s.meta.bunit_data), 
                                      wcs=s.meta.wcs)
    sc._meta = {}
    specs2d.append(sc)

ValueError: Missing input arguments - expected 2, got 1

In [19]:
mv0 = jdaviz.Mosviz()
mv0.load_data(specs1d, specs2d)